В этой тетрадке я предпринимаю отчаянную попытку перевести на ржя через модель mt5

In [100]:
%pip install transformers sentencepiece
%pip install simpletransformers
%pip install pympi-ling
%pip install razdel

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [95]:
%pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.6 MB 3.1 MB/s 
     |████████████████████████████████| 831.4 MB 791 bytes/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.7.0 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires Pillow<=6.2.2, but you have pillow 8.3.1 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
enot-utils 1.0.2 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
enot-utils 1.0.2 requires tqdm>=4.50.0, but you have tqdm 4.49.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/pyt

In [ ]:
#!unzip Разметки.zip

In [1]:
#!g1.1
import pandas as pd
import numpy as np
import pympi
from razdel import tokenize
from sklearn.model_selection import train_test_split
from torchtext.legacy import data
import torchtext.vocab as vocab
import torch

Execute error: Servant g1.1 not allocated: Allocation was interrupted

In [172]:
#!g1.1
# check if cuda is on
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False

True

## Загрузка данных

Загружаем данные, потом делим их на трейн и тест и сохраняем в соответствующие два файла

In [2]:
#!g1.1
import re
import json
import pymystem3
import os, sys
sys.path.append('../')
from argparse import Namespace

from pyeaf.pyeaf import EAFReader
from pyeaf.text import VocabularyVectorizer, TextStemmer, RSLStemmer, GramBinarizer

In [109]:
#!g1.1
er = EAFReader(directory="Разметки")
er.load()

In [110]:
#!g1.1
sentences_rus = er.get_sentences(er.RUS)
sentences_rsl = er.get_sentences(er.RSL_R)

In [ ]:
#st = TextStemmer()

#stem_sentences_rus, gram_sentences_rus = st.stem(sentences_rus, gram=True)
#stem_sentences_rsl = list(map(RSLStemmer.stem_sentence, sentences_rsl))

In [113]:
sentences_rsl = [' '.join(sent).strip('%№') for sent in sentences_rsl]

In [115]:
sentences_rsl[:5]

['здравствуйте',
 'скоро 23 февраль праздник мужчина2 poss:3ps',
 '8 м-а-р-т праздник poss:3ps женщина',
 'мы отдых',
 'день другой:pl работать2 обычно']

In [ ]:
#rus_sentences, rsl_sentences = read_elan_files(".")

In [ ]:
#rsl_sentences = [sent.strip('№%').replace('%>%', ' ') for sent in rsl_sentences]

In [105]:
def save_data(train, test, format_save="csv"):
    """ Save rsl_sentences and rus_sentences to csv or json"""
    if format_save == 'json':
        with open('train.json', 'w', encoding='utf-8') as file:
            train.to_json(file, orient='records', lines=True, force_ascii=False)
        with open('test.json', 'w', encoding='utf-8') as file:
            test.to_json(file, orient='records', lines=True, force_ascii=False)
    elif format_save == 'csv':
        with open('train.csv', 'w', encoding='utf-8') as file:
            train.to_csv(file, index=False)
        with open('test.csv', 'w', encoding='utf-8') as file:
            test.to_csv(file, index=False)

In [116]:
raw_data = {'input_text': sentences_rus, 'target_text': sentences_rsl}
df = pd.DataFrame(raw_data, columns=['input_text', 'target_text'])
train, test = train_test_split(df, test_size=0.2)

save_data(train, test)

In [117]:
df.head()

,input_text,target_text
0,Здравствуйте!,здравствуйте
1,Скоро 23 февраля - мужской праздник.,скоро 23 февраль праздник мужчина2 poss:3ps
2,8 марта - женский праздник.,8 м-а-р-т праздник poss:3ps женщина
3,Мы отдыхаем.,мы отдых
4,В остальные дни работаем как обычно.,день другой:pl работать2 обычно


In [3]:
#!g1.1
generated_data = pd.read_csv('generated_stemed.csv', encoding='utf-8')

generated_data['stem_rus'] = generated_data['new_stem_sent_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['rsl'] = generated_data['new_stem_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['gram_rus'] = generated_data['new_gram_sent_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [5]:
#!g1.1

test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [6]:
#!g1.1
generated_data.head(2)

,new_stem_sent_rus,new_gram_sent_rus,new_stem_rsl,stem_rus,rsl,gram_rus
0,"['лягушка', 'самка', 'богомол', 'репчатый', 'л...","[['APRO', 'им', 'жен', 'ед'], ['им', 'S', 'од'...","['лягушка', 'самка', 'богомол', 'репчатый', 'л...","[лягушка, самка, богомол, репчатый, лук, косой...","[лягушка, самка, богомол, репчатый, лук, косой...","[[APRO, им, жен, ед], [им, S, од, жен, ед], [и..."
1,"['жена', 'самочка', 'перепел', 'подвесной', 'м...","[['APRO', 'им', 'жен', 'ед'], ['им', 'S', 'од'...","['жена', 'самочка', 'перепел', 'подвесной', 'м...","[жена, самочка, перепел, подвесной, мостик, на...","[жена, самочка, перепел, подвесной, мостик, на...","[[APRO, им, жен, ед], [им, S, од, жен, ед], [и..."


Теперь мы можем настроить логгирование и загрузить модельку для русского вот отсюда:

https://huggingface.co/cointegrated/rut5-small

Я беру именно эту, потому что там только русский язык и моделька маленькая, значит, моя память это потянет.

In [7]:
#!g1.1
import logging
#from simpletransformers.t5 import T5Model, T5Args
from transformers import T5ForConditionalGeneration, T5Tokenizer

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small")

INFO:filelock:Lock 140045516567696 acquired on /tmp/xdg_cache/huggingface/transformers/a70e10655dd799aac70d77cb3c0e2ed2dbc7e0e9e5a5fdcea95b424e5c7696fc.49f19f2d88f5168206d686d69bd0f6495942381027a2decd2658470f4e669fb9.lock
INFO:filelock:Lock 140045516567696 released on /tmp/xdg_cache/huggingface/transformers/a70e10655dd799aac70d77cb3c0e2ed2dbc7e0e9e5a5fdcea95b424e5c7696fc.49f19f2d88f5168206d686d69bd0f6495942381027a2decd2658470f4e669fb9.lock
INFO:filelock:Lock 140045510616464 acquired on /tmp/xdg_cache/huggingface/transformers/3adab6a14a1904da4b71c606c6f6821873350a59e838d9f18f0c1bd9a488dc7a.b02fba13e4584f40329b20be0bad88c2ef97b5fda7d9f4f69b1cb3a620568a86.lock
INFO:filelock:Lock 140045510616464 released on /tmp/xdg_cache/huggingface/transformers/3adab6a14a1904da4b71c606c6f6821873350a59e838d9f18f0c1bd9a488dc7a.b02fba13e4584f40329b20be0bad88c2ef97b5fda7d9f4f69b1cb3a620568a86.lock
INFO:filelock:Lock 140045510019856 acquired on /tmp/xdg_cache/huggingface/transformers/ca238a6842426ff4024b2f4e3

Тестируем, что модель загрузилась

In [ ]:
#!g1.1
text = 'Ехал Грека через реку, видит Грека в реке рак. '
inputs = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, top_p=0.95, num_return_sequences=10, 
        repetition_penalty=2.5,
        max_length=32,
    )
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Теперь достаем данные снова

In [122]:
#!g1.1
#train_df = pd.read_csv("train.csv").astype(str)
#test_df = pd.read_csv("test.csv").astype(str)

In [123]:
#!g1.1
#train_df['prefix'] = ''
#test_df['prefix'] = ''

In [37]:
#!g1.1
generated_data['input_text'] = generated_data['stem_rus'].apply(' '.join)
generated_data['target_text'] = generated_data['rsl'].apply(' '.join)

In [38]:
#!g1.1
test_data['input_text'] = test_data['stem_rus'].apply(' '.join)
test_data['target_text'] = test_data['rsl'].apply(' '.join)

In [39]:
#!g1.1
generated_data['prefix'] = ''
test_data['prefix'] = ''

In [40]:
#!g1.1
test_data.head()

,test_stem_rus,test_gram_rus,test_rsl,stem_rus,rsl,gram_rus,input_text,target_text,prefix
0,"['а', 'когда', 'дед', 'мороз', 'вернуться', 'д...","[['CONJ'], ['CONJ'], ['им', 'мн', 'S', 'муж', ...","['indx', 'дед.мороз', 'clf', 'clf.группа', 'cl...","[а, когда, дед, мороз, вернуться, домой, мален...","[indx, дед.мороз, clf, clf.группа, clf.перемес...","[[CONJ], [CONJ], [им, мн, S, муж, од], [им, мн...",а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...,
1,"['а', 'мачеха', 'лишать', 'я', 'весь', 'радость']","[['CONJ'], ['им', 'S', 'од', 'жен', 'ед'], ['и...","['мама', 'второй', '3ps', 'все', 'радость', 'д...","[а, мачеха, лишать, я, весь, радость]","[мама, второй, 3ps, все, радость, давать, 1ps,...","[[CONJ], [им, S, од, жен, ед], [изъяв, V, пе, ...",а мачеха лишать я весь радость,мама второй 3ps все радость давать 1ps ноль,
2,"['вообще', 'я', 'думать', 'что', 'мочь', 'быть...","[['вводн', 'ADV'], ['им', 'ед', '1-л', 'SPRO']...",[],"[вообще, я, думать, что, мочь, быть, в, следую...",[],"[[вводн, ADV], [им, ед, 1-л, SPRO], [изъяв, V,...",вообще я думать что мочь быть в следующий год ...,,
3,"['проходить', 'мимо', 'мама', 'проводить', 'ки...","[['V', 'несов', 'деепр', 'пе', 'непрош'], ['PR...",[],"[проходить, мимо, мама, проводить, кисточка, с...",[],"[[V, несов, деепр, пе, непрош], [PR], [им, S, ...",проходить мимо мама проводить кисточка свой хв...,,
4,"['зачем', 'я', 'хотеть', 'изменять', 'свой', '...","[['ADVPRO'], ['им', 'ед', '1-л', 'SPRO'], ['из...","['зачем', 'желание', 'изменение', 'все4', 'при...","[зачем, я, хотеть, изменять, свой, привычка, т...","[зачем, желание, изменение, все4, привычка, тр...","[[ADVPRO], [им, ед, 1-л, SPRO], [изъяв, V, пе,...",зачем я хотеть изменять свой привычка тратить ...,зачем желание изменение все4 привычка тратить ...,


Переходим к обучению

Зафайнтьюненная модель будет сохранена в папке `outputs` по дефолту. (https://simpletransformers.ai/docs/tips-and-tricks/#saving-checkpoints)

In [41]:
#!g1.1
from simpletransformers.t5 import T5Model, T5Args

use_cuda = False

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 30
model_args.eval_batch_size = 30
model_args.num_train_epochs = 10
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Russian-RSL Translation"

model2 = T5Model("mt5", "cointegrated/rut5-small", args=model_args)

In [ ]:
# encodings_train = tokenizer.prepare_seq2seq_batch(list(train_df['input_text']), list(train_df['target_text']))

In [ ]:
# encodings_train.keys()

In [ ]:
# print(len(encodings_train['input_ids']), len(train_df))

In [42]:
#!g1.1
# Train the model
model2.train_model(generated_data, eval_data=test_data)

Training loss,1.31802
lr,0.001
global_step,1550
_runtime,412
_timestamp,1649222368
_step,30


Training loss,█▅▅▄▄▄▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▁▂
lr,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██████████
global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███




Using Adafactor for T5























INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_model: Training started
wandb: wandb version 0.12.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/
INFO:simpletransformers.t5.t5_utils: Creating features fr

(18920,
 {'global_step': [1892,
   3784,
   5676,
   7568,
   9460,
   11352,
   13244,
   15136,
   17028,
   18920],
  'eval_loss': [2.8038442251158924,
   3.4544577365968285,
   3.9132438927161983,
   4.366453932552803,
   4.560958868119775,
   4.850114101316871,
   4.997898596089061,
   5.116769976732208,
   5.163917832258271,
   5.342278329337516],
  'train_loss': [0.935966968536377,
   0.5932716727256775,
   0.37278836965560913,
   0.19077400863170624,
   0.18507827818393707,
   0.06183665990829468,
   0.17069508135318756,
   0.07266747206449509,
   0.03491602838039398,
   0.028819985687732697]})

Вот, что-то обучили...

Попробовала на 5, 15 и 10 эпох. На 5 эпох получились самые лучшие результаты.

P.S. Забыла про дурацкую токенизацию ржя, пришлось еще раз переобучить, в этот раз сделала на 7 эпох.

Теперь можно оценить и протестировать (пока тестирую модель на 10)

In [129]:
%pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [43]:
#!g1.1
import logging
import sacrebleu
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

model2 = T5Model("mt5", "outputs", args=model_args)

Попробуем оценить, что произошло

In [44]:
#!g1.1
truth_rsl = test_data["target_text"].tolist()
to_rsl = test_data["input_text"].tolist()

In [45]:
#!g1.1
rsl_preds = model2.predict(to_rsl)
rus_rsl_bleu = sacrebleu.corpus_bleu(rsl_preds, [truth_rsl])
print("--------------------------")
print("Russian to RSL: ", rus_rsl_bleu.score)


/home/jupyter/.local/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


EOFError: No valid references for a sentence!

In [ ]:
rsl_preds[:10]

In [ ]:
truth_rsl[0][:10]

**Результат:**

Russian to RSL (на неправильльной токенизации ржя):  28.527910962357836

Но как бы сравнить не с чем. Хотя чиселка вроде немаленькая такая

Но при этом почему-то, когда я переехала на более корректную предобработку русского, качество модели резко упало

Russian to RSL:  5.3837717877066265

In [ ]:
for x, y in zip(rsl_preds[:2], truth_rsl[0][:2]):
    print(x + '\t' + '===' + '\t' + y)

In [ ]:
rsl_preds[0], truth_rsl[0][0]

In [ ]:
with open("predict_10.tsv", "w", encoding="utf-8") as f:
    f.write('Russian' + '\t' + 'Predicted' + '\t' + 'Truth' + '\n')
    for rus, preds, real in zip(to_rsl, rsl_preds, truth_rsl[0]):
        f.write(rus + '\t' + preds + '\t' + real + '\n')